# Lattice v0.3 — Phase 3 Demo

This notebook demonstrates the Lattice pipeline with **Phase 3** features:

- **7-key field spec** — `prompt`, `type`, `format`, `enum`, `examples`, `bad_examples`, `default`
- **Dynamic prompt builder** — markdown headers + XML data boundaries (GPT-4.1 cookbook)
- **Default enforcement** — LLM refusals replaced with field `default` values in Python
- **FieldSpec validation** — unknown keys rejected at construction time
- **Model default** — `gpt-4.1-mini` (was nano)

Plus everything from Phase 2: cost tracking, error handling, multi-step pipelines, progress bars.

> **Note:** Jupyter runs its own async event loop, so we use `await pipeline.run_async(df)`.
> In scripts, use `pipeline.run(df)` (sync wrapper) instead.

In [1]:
import pandas as pd
from lattice import Pipeline, LLMStep, FunctionStep, EnrichmentConfig, FieldSpec

/Users/matthew.house/Programming/lattice/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Simple: One LLM step with shorthand fields

The simplest usage — 3 rows, 2 fields, one API call per row.

Default model is now `gpt-4.1-mini` ($0.40/$1.60 per 1M tokens). This cell costs ~$0.002.

In [2]:
df = pd.DataFrame({
    "company": ["Stripe", "Notion", "Figma"],
    "description": [
        "Online payment processing for internet businesses",
        "All-in-one workspace for notes, docs, and project management",
        "Collaborative interface design tool for teams",
    ],
})
df

,company,description
0,Stripe,Online payment processing for internet businesses
1,Notion,"All-in-one workspace for notes, docs, and proj..."
2,Figma,Collaborative interface design tool for teams


In [3]:
pipeline = Pipeline([
    LLMStep("analyze", fields={
        "category": "Classify into one of: Fintech, Productivity, Design, Other",
        "target_market": "Describe the primary target market in 10 words or less",
    })
])

result = await pipeline.run_async(df)
print(f"Model: gpt-4.1-mini (default) | Tokens: {result.cost.total_tokens}")
result.data

Pipeline: 100%|██████████| 1/1 [00:02<00:00,  2.11s/step, step=analyze]

Model: gpt-4.1-mini (default) | Tokens: 938


,company,description,category,target_market
0,Stripe,Online payment processing for internet businesses,Fintech,Internet businesses needing online payment pro...
1,Notion,"All-in-one workspace for notes, docs, and proj...",Productivity,Individuals and teams needing organized worksp...
2,Figma,Collaborative interface design tool for teams,Design,Teams needing collaborative interface design t...


In [4]:
# Cost and error summary
print(f"Success rate: {result.success_rate:.0%}")
print(f"Errors: {len(result.errors)}")
print(f"Total tokens: {result.cost.total_tokens}")
print(f"\nPer-step breakdown:")
for step_name, usage in result.cost.steps.items():
    print(f"  {step_name}: {usage.total_tokens} tokens ({usage.rows_processed} rows, model={usage.model})")

Success rate: 100%
Errors: 0
Total tokens: 938

Per-step breakdown:
  analyze: 938 tokens (3 rows, model=gpt-4.1-mini)


## 2. Full 7-key field spec — enum, examples, bad_examples, default

This is the core Phase 3 feature. Each field can use up to 7 keys:
- `prompt` (required) — the extraction instruction
- `type` — String, Number, Boolean, Date, List[String], JSON
- `format` — output format pattern
- `enum` — constrained value list (LLM MUST pick one)
- `examples` — good output examples
- `bad_examples` — anti-patterns to avoid
- `default` — fallback when data is insufficient (enforced in Python, not by the LLM)

In [5]:
pipeline = Pipeline([
    LLMStep("enrich", fields={
        "sector": {
            "prompt": "Classify the company's primary sector",
            "enum": ["Fintech", "Productivity", "Design", "Infrastructure", "Other"],
            "examples": ["Fintech", "Productivity"],
            "bad_examples": ["Tech company", "Software"],  # too vague
            "default": "Other",
        },
        "employee_count": {
            "prompt": "Estimate the number of employees",
            "type": "Number",
            "format": "integer",
            "default": 0,
        },
        "growth_stage": {
            "prompt": "Classify the company's growth stage based on its description and market position",
            "enum": ["Seed", "Growth", "Mature", "Decline"],
            "examples": ["Growth - rapidly expanding market share"],
            "default": "Unknown",
        },
    })
])

result = await pipeline.run_async(df)
result.data[["company", "sector", "employee_count", "growth_stage"]]

Pipeline: 100%|██████████| 1/1 [00:00<00:00,  1.48step/s, step=enrich]


,company,sector,employee_count,growth_stage
0,Stripe,Fintech,6000.0,Mature
1,Notion,Productivity,500.0,Growth
2,Figma,Design,1000.0,Growth


In [6]:
# Inspect the generated system prompt to see the dynamic builder in action
from lattice.steps.prompt_builder import build_system_message

step = pipeline.get_step("enrich")
sample_prompt = build_system_message(
    field_specs=step._field_specs,
    row={"company": "Stripe", "description": "Online payment processing"},
)
print(sample_prompt)

# Role
You are a structured data enrichment engine. Given one input row, a set of field specifications, and optional context from prior processing steps, produce a JSON object with exactly the requested fields as keys.

# Field Specification Keys
Each field below is described using these keys:
- **prompt**: the extraction instruction for this field
- **type**: expected output data type (String, Number, Boolean, Date, List[String], JSON)
- **format**: output format pattern to follow
- **enum**: constrained value list — the output MUST be one of these options exactly
- **examples**: good output examples showing expected style
- **bad_examples**: anti-patterns to avoid
- **default**: last-resort fallback if you truly cannot determine the value

# Output Rules
- Return ONLY a single valid JSON object. No prose, no code fences, no explanations.
- Top-level keys MUST be exactly: sector, employee_count, growth_stage
- Keep outputs concise and information-dense.
- For enum fields, the value MU

## 3. Default enforcement demo

When the LLM can't determine a value, it returns refusal text like "Unable to determine".
Phase 3's default enforcement catches this in Python and replaces it with the field's `default`.

We'll use a dataset with a made-up company to trigger refusals.

In [7]:
df_unknown = pd.DataFrame({
    "company": ["Stripe", "Xylophonica Dynamics Ltd"],
    "description": [
        "Online payment processing for internet businesses",
        "",  # no description — should trigger refusals
    ],
})

pipeline = Pipeline([
    LLMStep("analyze", fields={
        "sector": {
            "prompt": "What sector does this company operate in?",
            "enum": ["Fintech", "SaaS", "Hardware", "Other"],
            "default": "Unknown",
        },
        "founded_year": {
            "prompt": "What year was this company founded?",
            "type": "String",
            "format": "YYYY",
            "default": "N/A",
        },
    })
])

result = await pipeline.run_async(df_unknown)
result.data[["company", "sector", "founded_year"]]

Pipeline: 100%|██████████| 1/1 [00:00<00:00,  1.37step/s, step=analyze]


,company,sector,founded_year
0,Stripe,Fintech,2010
1,Xylophonica Dynamics Ltd,Other,N/A


## 4. FieldSpec validation — fail fast on bad specs

Unknown keys are rejected at LLMStep construction time (not at runtime).
This catches typos and legacy keys like `instructions` immediately.

In [8]:
from pydantic import ValidationError

# This SHOULD fail — "instructions" is not a valid key (use "prompt" instead)
try:
    LLMStep("bad", fields={
        "f1": {"prompt": "test", "instructions": "extra guidance"},
    })
except ValidationError as e:
    print("Caught validation error (as expected):")
    print(e.errors()[0]["type"], "—", e.errors()[0]["msg"])

Caught validation error (as expected):
extra_forbidden — Extra inputs are not permitted


## 5. Multi-step pipeline with FunctionStep + LLMStep

FunctionStep generates context, LLMStep uses it via `depends_on`. Internal `__` fields are filtered from output.

In [9]:
def generate_context(ctx):
    """Simulate an API lookup."""
    fake_data = {
        "Stripe": "Founded 2010. $95B valuation. 8000+ employees. Competes with Adyen, Square.",
        "Notion": "Founded 2013. $10B valuation. 500+ employees. Competes with Confluence, Coda.",
        "Figma": "Founded 2012. Acquired by Adobe for $20B (cancelled). Competes with Sketch, Canva.",
    }
    company = ctx.row["company"]
    return {"__context": fake_data.get(company, "No data available")}


pipeline = Pipeline([
    FunctionStep("lookup", fn=generate_context, fields=["__context"]),
    LLMStep("synthesize", fields={
        "competitive_position": {
            "prompt": "Rate the company's competitive position based on the context",
            "enum": ["Leader", "Challenger", "Niche"],
            "examples": ["Leader — dominant market share with strong moat"],
        },
        "investment_thesis": {
            "prompt": "One-sentence investment thesis using context and description",
            "examples": ["Strong moat in payments with expanding platform revenue"],
        },
    }, depends_on=["lookup"]),
])

result = await pipeline.run_async(df)
print("Columns:", list(result.data.columns))
assert "__context" not in result.data.columns  # internal fields filtered
result.data[["company", "competitive_position", "investment_thesis"]]

Pipeline: 100%|██████████| 2/2 [00:00<00:00,  2.10step/s, step=synthesize] 

Columns: ['company', 'description', 'competitive_position', 'investment_thesis']


,company,competitive_position,investment_thesis
0,Stripe,Leader,Dominant online payments platform with strong ...
1,Notion,Leader,Leading all-in-one workspace with strong marke...
2,Figma,Leader,Leading collaborative interface design tool wi...
